In [50]:
import datetime
import os

import pandas as pd
import numpy as np
# import psycopg2 
import sshtunnel
import sqlalchemy
from sqlalchemy.orm import Session
from sentence_transformers import SentenceTransformer

**Load data**

In [7]:
DATA_DIR = "./data"
FILE_NAME = "products_catalog.csv"

raw_df = pd.read_csv(os.path.join(DATA_DIR, FILE_NAME))

In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12491 entries, 0 to 12490
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ProductID     12491 non-null  int64 
 1   ProductName   12491 non-null  object
 2   ProductBrand  12491 non-null  object
 3   Gender        12491 non-null  object
 4   Price (INR)   12491 non-null  int64 
 5   NumImages     12491 non-null  int64 
 6   Description   12491 non-null  object
 7   PrimaryColor  11597 non-null  object
dtypes: int64(3), object(5)
memory usage: 780.8+ KB


In [9]:
raw_df.sample(3)

,ProductID,ProductName,ProductBrand,Gender,Price (INR),NumImages,Description,PrimaryColor
10078,10242895,Campus Men Grey Mesh Running Shoes,Campus,Men,799,5,"A pair of grey running sports shoes, has regul...",Grey
11559,10260451,Peter England Men Blue Checked Slim Fit Formal...,Peter England,Men,884,6,"Blue checked formal shirt, has a spread collar...",Blue
915,10006191,berrytree Girls Pink A-Line Organic Cotton Sus...,berrytree,Girls,599,6,"Pink printed knitted A-line dress, has a round...",Pink


### Converting `ProductName` to embeddings

In [52]:
MODEL_PATH = "./embedding_model"

In [53]:
embedding_model = SentenceTransformer(MODEL_PATH) #"all-mpnet-base-v2"

In [44]:
# embedding_model.save(MODEL_PATH)

In [43]:
%%time

product_name_embeddings = embedding_model.encode(raw_df["ProductName"].values)

CPU times: user 19min 48s, sys: 2.31 s, total: 19min 50s
Wall time: 5min 24s


In [10]:
EMBEDDING_FILE_NAME = "product_name_embeddings.pk"

In [14]:
import pickle

# with open("./product_name_embeddings.pk", "wb") as f:
#     pickle.dump(product_name_embeddings, f)
    
with open(os.path.join(DATA_DIR, EMBEDDING_FILE_NAME), "rb") as f:
    product_name_embeddings = pickle.load(f)
    
product_name_embeddings.shape

(12491, 768)

In [15]:
product_name_embeddings_dict = [
    {"ProductID":raw_df["ProductID"].values[i],"embeddings":product_name_embeddings[i]} for i in range(len(product_name_embeddings))
]

### Uploading data to feature store

In [23]:
df = raw_df.merge(
    pd.DataFrame(product_name_embeddings_dict),
    on="ProductID",
    how="left"
)

df.shape

(12491, 9)

<font color="blue">I was using Aurora Serverless postgres db with pgvector extension on AWS for vector storage and retrieval. Aurora Serverless cannot be access from outside VPC, so I setup a bastion server, and use SSHTunnel to connect to db via bastion server.</font>

In [32]:
ec2_url = ""

dbhost = ""
dbport = 1111
dbuser = ""
dbpass = ""

server = sshtunnel.SSHTunnelForwarder(
    ssh_address_or_host=(ec2_url, 22),
    ssh_username="",
    ssh_pkey="",
    remote_bind_address=(dbhost, dbport),
    local_bind_address=("localhost", 5433),
)

**Table (or model)**

In [30]:
from pgvector.sqlalchemy import Vector
import sqlalchemy
from sqlalchemy.orm import DeclarativeBase,mapped_column
from sqlalchemy import Integer, String, TIMESTAMP, Float

EMBEDDINGS_LENGTH = df["embeddings"].iloc[0].shape[0]

class Base(DeclarativeBase):
    pass

class Products(Base):
    __tablename__ = "products"
    __table_args__ = {'extend_existing': True}

    pid = mapped_column(Integer, primary_key=True)
    pname = mapped_column(String)
    brand = mapped_column(String)
    gender = mapped_column(String)
    price = mapped_column(Float)
    n_images = mapped_column(Integer)
    description = mapped_column(String)
    color = mapped_column(String)
    embeddings = mapped_column(Vector(EMBEDDINGS_LENGTH))
    added_timestamp = mapped_column(TIMESTAMP)

In [ ]:
server.start()

engine = sqlalchemy.create_engine(
    f"""postgresql+psycopg2://{dbuser}:"""
    f"""{dbpass}@{server.local_bind_host}:"""
    f"""{server.local_bind_port}/feature_store""",
    echo=False,
)

In [34]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

2023-11-02 23:17:16,535 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-02 23:17:16,539 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:17:17,369 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-02 23:17:17,371 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:17:18,175 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-02 23:17:18,177 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:17:18,935 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-02 23:17:18,951 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

**Insert data in table**

In [40]:
df.columns.tolist()

['ProductID',
 'ProductName',
 'ProductBrand',
 'Gender',
 'Price (INR)',
 'NumImages',
 'Description',
 'PrimaryColor',
 'embeddings']

In [42]:
df_dict = [{"pid":rec["ProductID"],"pname":rec["ProductName"],"brand":rec["ProductBrand"],"gender":rec["Gender"],"price":rec["Price (INR)"],"n_images":rec["NumImages"],"description":rec["Description"],"color":rec["PrimaryColor"],"embeddings":rec["embeddings"],"added_timestamp":datetime.datetime.now()} for rec in df.to_dict(orient="records")]

assert(len(df_dict) == len(df))
len(df_dict)

12491

In [45]:
with Session(engine) as session:
    session.execute(sqlalchemy.insert(Products), df_dict)
    session.commit()

2023-11-02 23:24:31,115 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-02 23:24:31,117 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:24:31,938 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-02 23:24:31,940 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:24:32,708 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-02 23:24:32,710 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:24:33,574 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-02 23:24:40,983 INFO sqlalchemy.engine.Engine INSERT INTO products (pid, pname, brand, gender, price, n_images, description, color, embeddings, added_timestamp) VALUES (%(pid__0)s, %(pname__0)s, %(brand__0)s, %(gender__0)s, %(price__0)s, %(n_images__0)s, %(description__0)s, %(color__0)s, %(embed ... 183670 characters truncated ... _images__999)s, %(description__999)s, %(color__999)s, %(embeddings__999)s, %(added_timestamp__999)s)
2023-11-02 23:24:40,983 INFO sqlalchemy

2023-11-02 23:25:02,821 INFO sqlalchemy.engine.Engine INSERT INTO products (pid, pname, brand, gender, price, n_images, description, color, embeddings, added_timestamp) VALUES (%(pid__0)s, %(pname__0)s, %(brand__0)s, %(gender__0)s, %(price__0)s, %(n_images__0)s, %(description__0)s, %(color__0)s, %(embed ... 183670 characters truncated ... _images__999)s, %(description__999)s, %(color__999)s, %(embeddings__999)s, %(added_timestamp__999)s)
2023-11-02 23:25:02,823 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/13 (unordered)] {'gender__0': 'Men', 'description__0': 'Beige self-design formal shirt, has a spread collar, long sleeves, button placket,  straight hem, one patch pocket', 'brand__0': 'Raymond', 'color__0': ' Beige', 'embeddings__0': '[-0.008026523515582085,0.030790258198976517,0.010533036664128304,0.04654913768172264,-0.0791148766875267,-0.018506985157728195,-0.012486363761126995, ... (15928 characters truncated) ... 0.02914738468825817,-0.0013721681898459792,0.0253192875534296

2023-11-02 23:25:14,845 INFO sqlalchemy.engine.Engine INSERT INTO products (pid, pname, brand, gender, price, n_images, description, color, embeddings, added_timestamp) VALUES (%(pid__0)s, %(pname__0)s, %(brand__0)s, %(gender__0)s, %(price__0)s, %(n_images__0)s, %(description__0)s, %(color__0)s, %(embed ... 183670 characters truncated ... _images__999)s, %(description__999)s, %(color__999)s, %(embeddings__999)s, %(added_timestamp__999)s)
2023-11-02 23:25:14,846 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/13 (unordered)] {'gender__0': 'Women', 'description__0': 'Display: AnalogueMovement: QuartzPower source: BatteryDial style: Solid round metal dialFeatures: Reset TimeStrap style: Black regular, leather strap with a tang closureWater resistance: 30 mWarranty: 2 yearsWarranty provided by brand/manufacturer Comes in a signature Titan case', 'brand__0': 'Titan', 'color__0': ' Black', 'embeddings__0': '[0.0008280155016109347,-0.008397857658565044,-0.004477004986256361,0.04591348022222

2023-11-02 23:25:28,452 INFO sqlalchemy.engine.Engine INSERT INTO products (pid, pname, brand, gender, price, n_images, description, color, embeddings, added_timestamp) VALUES (%(pid__0)s, %(pname__0)s, %(brand__0)s, %(gender__0)s, %(price__0)s, %(n_images__0)s, %(description__0)s, %(color__0)s, %(embed ... 183670 characters truncated ... _images__999)s, %(description__999)s, %(color__999)s, %(embeddings__999)s, %(added_timestamp__999)s)
2023-11-02 23:25:28,454 INFO sqlalchemy.engine.Engine [insertmanyvalues 4/13 (unordered)] {'gender__0': 'Unisex', 'description__0': 'Set content: Single Door curtainColour: BeigePattern: Floral DoorLight blocking: RegularFitting: Eyelet', 'brand__0': 'VRINDA', 'color__0': ' Beige', 'embeddings__0': '[0.007669719401746988,-0.020761268213391304,-0.0012162966886535287,-0.013211172074079514,-0.0664496198296547,-0.06213679909706116,-0.0449892207980155 ... (15898 characters truncated) ... 0.0018716444028541446,-0.047862667590379715,0.05416778847575188,-0.030

2023-11-02 23:25:42,345 INFO sqlalchemy.engine.Engine INSERT INTO products (pid, pname, brand, gender, price, n_images, description, color, embeddings, added_timestamp) VALUES (%(pid__0)s, %(pname__0)s, %(brand__0)s, %(gender__0)s, %(price__0)s, %(n_images__0)s, %(description__0)s, %(color__0)s, %(embed ... 183670 characters truncated ... _images__999)s, %(description__999)s, %(color__999)s, %(embeddings__999)s, %(added_timestamp__999)s)
2023-11-02 23:25:42,346 INFO sqlalchemy.engine.Engine [insertmanyvalues 5/13 (unordered)] {'gender__0': 'Women', 'description__0': 'A pair of round-toe red solid slip-on sneakers, has regular styling, slip-on detailSynthetic Leather upperCushioned footbedTextured and patterned outsole', 'brand__0': 'FAUSTO', 'color__0': ' Red', 'embeddings__0': '[0.008745112456381321,-0.02371208742260933,0.0013677276438102126,0.02258233353495598,0.061441946774721146,0.024396570399403572,-0.006567405536770821, ... (15929 characters truncated) ... 89,-0.04575112089514732

...

2023-11-02 23:26:34,347 INFO sqlalchemy.engine.Engine [insertmanyvalues 10/13 (unordered)] {'gender__0': 'Girls', 'description__0': 'Red solid Open Front Jacket, has a spread collar, 2 pockets, button closure, long sleeves, hem with toggle hem', 'brand__0': 'U.S. Polo Assn. Kids', 'color__0': ' Red', 'embeddings__0': '[0.009479990229010582,-0.042210109531879425,0.003233256982639432,0.05335729196667671,0.05559425428509712,-0.0035138556268066168,0.007834509015083313, ... (15921 characters truncated) ... 605,0.0244683176279068,0.01346330251544714,0.10057403147220612,-0.011585800908505917,0.01226053200662136,-0.003451623022556305,-0.014940154738724232]', 'n_images__0': 5, 'price__0': 1649, 'pid__0': 10213231, 'added_timestamp__0': datetime.datetime(2023, 11, 2, 23, 23, 54, 302113), 'pname__0': 'U.S. Polo Assn. Kids Girls Red Solid Open Front Jacket', 'gender__1': 'Women', 'description__1': 'Olive Green solid zip around wallet, has a wrist loop1 main compartment, a separator sleeve and a zi

2023-11-02 23:26:43,024 INFO sqlalchemy.engine.Engine [insertmanyvalues 11/13 (unordered)] {'gender__0': 'Men', 'description__0': 'The classic, one-and-doneSmooth, soft poplin weave with stretchLong sleeves with button cuffsButton collar, button frontChest patch pocketCurved shirt tail hemRear pleat at back yoke', 'brand__0': 'GAP', 'color__0': nan, 'embeddings__0': '[-0.035365018993616104,0.025198213756084442,-0.011756335385143757,0.007143646478652954,-0.05270123854279518,-0.015274775214493275,0.00922527257353067 ... (15954 characters truncated) ... 65,-0.0008394002798013389,0.025223495438694954,0.025663577020168304,0.05831265076994896,0.02483483962714672,0.02220909483730793,0.016015540808439255]', 'n_images__0': 5, 'price__0': 1999, 'pid__0': 10236507, 'added_timestamp__0': datetime.datetime(2023, 11, 2, 23, 23, 54, 303894), 'pname__0': "GAP Men's Checked Lived-In Stretch Poplin Shirt", 'gender__1': 'Girls', 'description__1': 'Standout, specially designed graphics let you be youSoft 

2023-11-02 23:26:51,204 INFO sqlalchemy.engine.Engine [insertmanyvalues 12/13 (unordered)] {'gender__0': 'Women', 'description__0': 'Off-white printed T-shirt, has a round neck, and short sleeves', 'brand__0': 'Ginger by Lifestyle', 'color__0': ' White', 'embeddings__0': '[0.03398875519633293,-0.017770588397979736,-0.018633490428328514,0.012555263936519623,-0.025729894638061523,0.014666218310594559,-0.00552805466577410 ... (15911 characters truncated) ... ,-0.0012905055191367865,0.003872204339131713,0.06439074873924255,0.004592626355588436,0.02736963890492916,-0.04005208984017372,-0.026844941079616547]', 'n_images__0': 4, 'price__0': 249, 'pid__0': 10251279, 'added_timestamp__0': datetime.datetime(2023, 11, 2, 23, 23, 54, 305637), 'pname__0': 'Ginger by Lifestyle Women Off-White Printed Round Neck T-shirt', 'gender__1': 'Women', 'description__1': 'Special Technology:SoftFoam+: Cushioned foot bed molds to foot and provides long-lasting step-in comfortDesign Details:A pair of black runni

2023-11-02 23:27:01,152 INFO sqlalchemy.engine.Engine INSERT INTO products (pid, pname, brand, gender, price, n_images, description, color, embeddings, added_timestamp) VALUES (%(pid__0)s, %(pname__0)s, %(brand__0)s, %(gender__0)s, %(price__0)s, %(n_images__0)s, %(description__0)s, %(color__0)s, %(embed ... 89505 characters truncated ... _images__490)s, %(description__490)s, %(color__490)s, %(embeddings__490)s, %(added_timestamp__490)s)
2023-11-02 23:27:01,154 INFO sqlalchemy.engine.Engine [insertmanyvalues 13/13 (unordered)] {'gender__0': 'Women', 'description__0': 'Silver-plated and brown oxidised choker necklaceMaterial: BrassClosure: Drawstring', 'brand__0': 'Studio Voylla', 'color__0': ' Silver', 'embeddings__0': '[0.03860096633434296,-0.017212416976690292,0.02027895115315914,0.023260440677404404,-0.06281744688749313,0.01755242608487606,0.028972279280424118,0.0 ... (15924 characters truncated) ... 0.020726582035422325,-0.008267114870250225,0.0028507686220109463,0.04915731400251388

2023-11-02 23:27:13,856 INFO sqlalchemy.engine.Engine COMMIT


In [48]:
with Session(engine) as session:
    stmt = sqlalchemy.text("SELECT count(*) from products;")
    stmt_response = session.scalars(stmt).first()
    
stmt_response

2023-11-02 23:30:56,542 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-11-02 23:30:56,544 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:30:57,474 INFO sqlalchemy.engine.Engine select current_schema()
2023-11-02 23:30:57,476 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:30:58,180 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-11-02 23:30:58,181 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-11-02 23:30:59,002 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-02 23:30:59,005 INFO sqlalchemy.engine.Engine SELECT count(*) from products;
2023-11-02 23:30:59,007 INFO sqlalchemy.engine.Engine [generated in 0.00255s] {}
2023-11-02 23:30:59,864 INFO sqlalchemy.engine.Engine ROLLBACK


[12491]

**Getting recommendations**

In [80]:
server.start()

engine = sqlalchemy.create_engine(
    f"""postgresql+psycopg2://{dbuser}:"""
    f"""{dbpass}@{server.local_bind_host}:"""
    f"""{server.local_bind_port}/feature_store""",
    echo=False,
)

In [81]:
query_emb = embedding_model.encode("dress for office party for women")

In [82]:
with Session(engine) as session:
    stmt = sqlalchemy.select(Products.pid).order_by(Products.embeddings.l2_distance(query_emb)).limit(10)
    stmt_response = session.scalars(stmt).all()

In [83]:
stmt_response

[10273987,
 10274029,
 10233311,
 10233281,
 10086107,
 10015997,
 10207393,
 10082953,
 10051325,
 10189591]

In [84]:
raw_df.loc[
    raw_df["ProductID"].isin(stmt_response),
    "ProductName"
].values

array(['Tokyo Talkies Women Black & Red Floral Print Fit and Flare Dress',
       'Tokyo Talkies Women Navy Blue Sheath Dress',
       'Tokyo Talkies Women Green Floral Print Fit and Flare Dress',
       'Tokyo Talkies Women Black & White Fit and Flare Dress',
       'Tokyo Talkies Women Pink & Blue Floral Print Fit and Flare Dress',
       'W Women Green & Peach-Coloured Printed Maxi Dress',
       'AND Women Navy Embellished Maxi Dress',
       'AND Women Navy Blue Floral Printed Fit and Flare Dress',
       'ONLY Women Red & Black Shirt Dress',
       'ONLY Women Green & Black Shirt Dress'], dtype=object)

In [86]:
server.stop()
engine.dispose()

***